In [13]:
import os
import pickle

# Define the relative path from the current working directory
relative_folder_path = "week5/shared"
file_name = "performer_network.pkl"

# Use the relative path to construct the full path
file_path = os.path.join(relative_folder_path, file_name)

# Load the graph from the specified file path
with open(file_path, 'rb') as f:
    G = pickle.load(f)

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")


Number of nodes: 1939
Number of edges: 17578
